#  Practica de limpieza de texto con pandas - Solución

#### Por Jose R. Zapata - https://joserzapata.github.io/


> **Objetivo**: practicar métodos de `pandas.Series.str` para limpiar y normalizar texto en DataFrames.
> Cada ejercicio incluye un **Enunciado** y una celda donde se ve el resultado de la solución.



**Referencias:**
- Procesamiento básico de texto (curso de NLP): https://joserzapata.github.io/courses/nlp/procesamiento-basico/
- Documentación de `pandas.Series.str`: https://pandas.pydata.org/docs/reference/series.html#string-handling


## Instalación de dependencias

instala e importa las librerias que necesites para ejecutar los ejercicios, por ejemplo para instalar Jupyter usa:

```bash
uv add jupyter
```


In [154]:
# Copie su código aca
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_colwidth", None)

## Conjuntos de datos de juguete
Creamos 4 DataFrames para practicar distintas tareas de limpieza: texto general, tweets, productos y personas.

In [155]:
## Texto general
df_texto = pd.DataFrame(
    {
        "texto": [
            "  ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)  ",
            "Pandas > numpy? 🤔  Email: persona@example.com   ",
            "Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan",
            "Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123",
            "      TABLAS\t, \n espacios   y saltos de línea.\r\n",
            "Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44",
            "Dirección: Cll 10 # 5-20; Medellín. Barrio: La América",
            "Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",
        ]
    }
)

## Tweets
df_tweets = pd.DataFrame(
    {
        "tweet": [
            "RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python",
            "¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊",
            "Probando cosas en Jupyter... sin link ni hashtag",
            "@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml",
            "¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data",
        ]
    }
)

## Productos y precios
df_productos = pd.DataFrame(
    {
        "producto": [
            "Camisa talla M",
            "Pantalón-XL",
            "Zapato, Talla: 42",
            "Blusa s",
            "Medias 10-12",
            "Polo Talla l",
            "Vestido - 36",
            "Sombrero (talla Única)",
        ],
        "precio": [
            "$1.234,50",
            "USD 45",
            "30,00 €",
            "25.000",
            "$ 0",
            "S/. 120.90",
            "COP 9.990",
            "AR$ 2.550,00",
        ],
    }
)

## Personas, respuestas y direcciones
df_personas = pd.DataFrame(
    {
        "nombre": [
            "ana María LOPEZ",
            "Juan  perez",
            "Ñandú   Gómez",
            "Miguel (Soporte)",
            "  MÓNICA de la CRUZ  ",
            "luis-delgado",
        ],
        "categoria": ["Sí", "si", "SI ", "No", "—", None],
        "direccion": [
            "Calle 45 # 12-34, Bogotá",
            "Av. Siempre Viva 742 - Lima",
            "Cll. 10 No. 5-20 Medellín",
            "Cra 7a # 45-60, Bogotá",
            "Av. 9 #12-34  Cali",
            "Av. Insurgentes Sur 1234, CDMX",
        ],
        "id_raw": [
            "abc-0001",
            "abc 001",
            "ABC0002",
            "Abc_003",
            "abc-00004",
            "ABC-0005",
        ],
    }
)

In [156]:
print("df_texto")
display(df_texto)
print("df_tweets")
display(df_tweets)
print("df_productos")
display(df_productos)
print("df_personas")
display(df_personas)

df_texto


,texto
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :)
1,Pandas > numpy? 🤔 Email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123
4,"TABLAS\t, \n espacios y saltos de línea.\r\n"
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


df_tweets


,tweet
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊
2,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data


df_productos


,producto,precio
0,Camisa talla M,"$1.234,50"
1,Pantalón-XL,USD 45
2,"Zapato, Talla: 42","30,00 €"
3,Blusa s,25.000
4,Medias 10-12,$ 0
5,Polo Talla l,S/. 120.90
6,Vestido - 36,COP 9.990
7,Sombrero (talla Única),"AR$ 2.550,00"


df_personas


,nombre,categoria,direccion,id_raw
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005


## Inspección rápida

- Explora los cuatro DataFrames
- Identifica qué columnas requieren **limpieza textual** y por qué (espacios, acentos, emojis, URLs, etc.).
- Escribe un breve comentario (como comentario en la celda) con tus observaciones.

In [157]:
# Observaciones: detección rápida de problemas de limpieza relevantes para NLP.
patrones = {
    "espacios_multiples": r"\s{2,}",
    "acentos": r"[ÁÉÍÓÚáéíóúÑñÜü]",
    "emojis": r"[\U0001F300-\U0001FAFF\U00002600-\U000027BF]",
    "urls": r"https?://\S+",
    "correos": r"[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+",
    "hashtags_menciones": r"[@#]",
}
for nombre_df, df in {
    "df_texto": df_texto,
    "df_tweets": df_tweets,
    "df_productos": df_productos,
    "df_personas": df_personas,
}.items():
    print(f"=== {nombre_df} ===")
    for columna in df.select_dtypes(include="object").columns:
        serie = df[columna].fillna("")
        hallazgos = []
        if (serie != serie.str.strip()).any():
            hallazgos.append("espacios_inicio_fin")
        for etiqueta, patron in patrones.items():
            if serie.str.contains(patron, regex=True).any():
                hallazgos.append(etiqueta)
        if hallazgos:
            print(f"- {columna} errores textuales: {', '.join(sorted(set(hallazgos)))}")
    print()

=== df_texto ===
- texto errores textuales: acentos, correos, emojis, espacios_inicio_fin, espacios_multiples, hashtags_menciones, urls

=== df_tweets ===
- tweet errores textuales: acentos, emojis, hashtags_menciones, urls

=== df_productos ===
- producto errores textuales: acentos

=== df_personas ===
- nombre errores textuales: acentos, espacios_inicio_fin, espacios_multiples
- categoria errores textuales: acentos, espacios_inicio_fin
- direccion errores textuales: acentos, espacios_multiples, hashtags_menciones



In [158]:
df_texto.info()
df_tweets.info()
df_productos.info()
df_personas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   8 non-null      object
dtypes: object(1)
memory usage: 196.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   producto  8 non-null      object
 1   precio    8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     6 n

## 1) Normaliza a minúsculas

En el DataFrame `df_texto`, crea una nueva columna `texto_min` que contenga el texto de la columna `texto` en minúsculas.

In [159]:
# Copie su código aca
df_texto["texto_min"] = df_texto["texto"].str.lower()

In [160]:
df_texto[["texto", "texto_min"]]


,texto,texto_min
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :)
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta!!! 3x2 en jabón líquido 500ml - código: a-123
4,"TABLAS\t, \n espacios y saltos de línea.\r\n","tablas\t, \n espacios y saltos de línea.\r\n"
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,dirección: cll 10 # 5-20; medellín. barrio: la américa
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 2) Eliminar Espacios

Genera  la columna `texto_espacios` eliminando espacios al inicio/fin

In [161]:
# Copie su código aca
df_texto["texto_espacios"] = df_texto["texto_min"].str.strip()

In [162]:
df_texto[["texto_espacios", "texto_min"]]


,texto_espacios,texto_min
0,¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :),¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :)
1,pandas > numpy? 🤔 email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan
3,oferta!!! 3x2 en jabón líquido 500ml - código: a-123,oferta!!! 3x2 en jabón líquido 500ml - código: a-123
4,"tablas\t, \n espacios y saltos de línea.","tablas\t, \n espacios y saltos de línea.\r\n"
5,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44
6,dirección: cll 10 # 5-20; medellín. barrio: la américa,dirección: cll 10 # 5-20; medellín. barrio: la américa
7,"emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 3) Puntuación

Crea `texto_sin_punct` removiendo puntuación y símbolos, conservando letras y espacios.

In [163]:
# Copie su código aca
df_texto["texto_sin_punct"] = df_texto["texto_espacios"].str.replace(
    r"[^\w\s]", "", regex=True
)

In [164]:
df_texto["texto_sin_punct"]


0    hola mundo  esto es   un ejemplo visita httpsmiwebcom  datascience  
1                                 pandas  numpy   email personaexamplecom
2             me gusta el café colombiano es buenísimo café colombia juan
3                          oferta 3x2 en jabón líquido 500ml  código a123
4                                tablas\t \n espacios   y saltos de línea
5                        teléfono 57 3001234567 whatsapp 57 300 222 33 44
6                        dirección cll 10  520 medellín barrio la américa
7                                           emoji test  símbolos  y otros
Name: texto_sin_punct, dtype: object

## 4) Acentos

Elimina los acentos de las vocales del texto `df_texto` y grábalo en una nueva columna `texto_sin_acentos`.

In [165]:
# Copie su código aca
def quitar_acentos(text: str) -> str:
    MAP_VOCALES = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u",
    }
    translate = str.maketrans(MAP_VOCALES)
    text = text.translate(translate)
    return text


df_texto["texto_sin_acentos"] = df_texto["texto_sin_punct"].apply(quitar_acentos)

In [166]:
df_texto["texto_sin_acentos"]


0    hola mundo  esto es   un ejemplo visita httpsmiwebcom  datascience  
1                                 pandas  numpy   email personaexamplecom
2             me gusta el cafe colombiano es buenisimo cafe colombia juan
3                          oferta 3x2 en jabon liquido 500ml  codigo a123
4                                tablas\t \n espacios   y saltos de linea
5                        telefono 57 3001234567 whatsapp 57 300 222 33 44
6                        direccion cll 10  520 medellin barrio la america
7                                           emoji test  simbolos  y otros
Name: texto_sin_acentos, dtype: object

## 5) Emojis

Crear la columna `texto_sin_emoji` quitando emojis, pictogramas y símbolos gráficos comunes.

In [167]:
# Copie su código aca
df_texto["texto_sin_emoji"] = df_texto["texto_sin_acentos"].str.replace(
    r"[\U0001F300-\U0001FAFF\U00002600-\U000027BF]", "", regex=True
)

In [168]:
df_texto["texto_sin_emoji"]

0    hola mundo  esto es   un ejemplo visita httpsmiwebcom  datascience  
1                                 pandas  numpy   email personaexamplecom
2             me gusta el cafe colombiano es buenisimo cafe colombia juan
3                          oferta 3x2 en jabon liquido 500ml  codigo a123
4                                tablas\t \n espacios   y saltos de linea
5                        telefono 57 3001234567 whatsapp 57 300 222 33 44
6                        direccion cll 10  520 medellin barrio la america
7                                           emoji test  simbolos  y otros
Name: texto_sin_emoji, dtype: object

## 6) Extrae emails y URLs

En `df_texto`, crea las columnas `email` y `urls` extrayendo emails y URLs del texto original.

**Patrones sugeridos:**
- Email: `[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+`
- URL: `https?://\S+`

In [169]:
# Copie su código aca
df_texto["email"] = df_texto["texto"].str.extract(
    r"([A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+)", expand=False
)
df_texto["urls"] = df_texto["texto"].str.extract(r"(https?://\S+)", expand=False)


In [170]:
df_texto[["texto", "email", "urls"]]


,texto,email,urls
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),NaN,https://miweb.com
1,Pandas > numpy? 🤔 Email: persona@example.com,persona@example.com,NaN
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,NaN,NaN
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,NaN,NaN
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",NaN,NaN
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,NaN,NaN
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,NaN,NaN
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",NaN,NaN


## 7) Hashtags, menciones y URLs en tweets

En `df_tweets`, crea las columnas `hashtags`, `mentions` y `urls`
extrayendo hashtags, menciones y URLs del texto del tweet.

In [171]:
# Copie su código aca
df_tweets["hashtags"] = df_tweets["tweet"].str.findall(r"#\w+")
df_tweets["mentions"] = df_tweets["tweet"].str.findall(r"@\w+")
df_tweets["urls"] = df_tweets["tweet"].str.extract(r"(https?://\S+)", expand=False)

In [172]:
df_tweets[["tweet", "hashtags", "mentions", "urls"]]

,tweet,hashtags,mentions,urls
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,"[#nlp, #python]",[@maria],http://blog.com/post?id=45
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,"[#datos, #Python]",[@data_science],https://example.org
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],NaN
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,"[#nlp, #ml]","[@juan, @ana]",https://cursos.ai
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,[#data],[],https://foro.com


## 8) ¿Es retuit? Limpia el prefijo

Crea la columna `es_rt` (booleano) si el tweet comienza con `RT`. Luego genera la columna `tweet_sin_rt` quitando el prefijo `RT @usuario:` del inicio.

In [173]:
# Copie su código aca
df_tweets["es_rt"] = df_tweets["tweet"].str.startswith("RT")
df_tweets["tweet_sin_rt"] = df_tweets.apply(
    lambda row: row["tweet"][3:].strip() if row["es_rt"] else row["tweet"], axis=1
)

In [174]:
df_tweets[["tweet", "hashtags", "mentions", "urls", "es_rt", "tweet_sin_rt"]]

,tweet,hashtags,mentions,urls,es_rt,tweet_sin_rt
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,"[#nlp, #python]",[@maria],http://blog.com/post?id=45,True,@maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,"[#datos, #Python]",[@data_science],https://example.org,False,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],NaN,False,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,"[#nlp, #ml]","[@juan, @ana]",https://cursos.ai,False,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,[#data],[],https://foro.com,False,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data


## 9) `tweet_limpio`

Crea `tweet_limpio` removiendo URLs, menciones y hashtags; además, elimina la puntuación sobrante, quita espacios y pasa a minúsculas.

In [175]:
# Copie su código aca


def limpiar_tweet(text: str) -> str:
    # Remover URLs
    text = re.sub(r"https?://\S+", "", text)
    # Remover menciones
    text = re.sub(r"@\w+", "", text)
    # Remover hashtags
    text = re.sub(r"#\w+", "", text)
    # Eliminar puntuación sobrante
    text = re.sub(r"[^\w\s]", "", text)
    # Quitar espacios adicionales
    text = " ".join(text.split())
    # Pasar a minúsculas
    text = text.lower()
    return text


df_tweets["tweet_limpio"] = df_tweets["tweet_sin_rt"].apply(limpiar_tweet)

In [176]:
df_tweets[["tweet", "tweet_limpio"]]

,tweet,tweet_limpio
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,nuevo post en el blog
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,me encanta pandas
2,Probando cosas en Jupyter... sin link ni hashtag,probando cosas en jupyter sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,y lanzaron curso de nlp en
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,pandas o polars debátanlo aquí


## 10) Talla en `df_productos`

Extrae y estandariza la talla:
- Letras: `XS, S, M, L, XL, XXL`, o `TU` (talla única).
- Números: captura como `talla_num`.
Crea la columna`talla_std` (letras) y `talla_num` (numérica).

In [177]:
# Copie su código aca

df_productos["talla_std"] = (
    df_productos["producto"]
    .str.extract(
        r"\b(XS|S|M|L|XL|XXL|TU|Única|Unica|ÚNICA|UNICA)\b",
        flags=re.IGNORECASE,
        expand=False,
    )
    .str.upper()
    .replace({"ÚNICA": "TU", "UNICA": "TU"})
)
df_productos["talla_num"] = df_productos["producto"].str.extract(
    r"(\d+(?:\.\d+)?)", expand=False
)

In [178]:
df_productos[["producto", "precio", "talla_std", "talla_num"]]

,producto,precio,talla_std,talla_num
0,Camisa talla M,"$1.234,50",M,NaN
1,Pantalón-XL,USD 45,XL,NaN
2,"Zapato, Talla: 42","30,00 €",NaN,42
3,Blusa s,25.000,S,NaN
4,Medias 10-12,$ 0,NaN,10
5,Polo Talla l,S/. 120.90,L,NaN
6,Vestido - 36,COP 9.990,NaN,36
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN


## 11) Extraer de precios

Convierte la columna `precio` a numérico (`precio_num`) independiente del formato (`$1.234,50`, `USD 45`, `30,00 €`, `COP 9.990`, etc.).

In [179]:
# Copie su código aca
def convertir_precio(s):
    if pd.isna(s):
        return np.nan
    s = str(s).replace("\xa0", " ").strip()
    s2 = re.sub(r"[^0-9,.-]", "", s)
    if "," in s2 and "." in s2:
        s2 = s2.replace(".", "").replace(",", ".")
    elif "," in s2 and "." not in s2:
        s2 = s2.replace(",", ".")
    s2 = s2.replace(",", "")
    try:
        return float(s2)
    except Exception:
        return np.nan


df_productos["precio_num"] = df_productos["precio"].apply(convertir_precio)


In [180]:
df_productos[["producto", "precio", "talla_std", "talla_num", "precio_num"]]

,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.50
1,Pantalón-XL,USD 45,XL,NaN,45.00
2,"Zapato, Talla: 42","30,00 €",NaN,42,30.00
3,Blusa s,25.000,S,NaN,25.00
4,Medias 10-12,$ 0,NaN,10,0.00
5,Polo Talla l,S/. 120.90,L,NaN,NaN
6,Vestido - 36,COP 9.990,NaN,36,9.99
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN,2550.00


## 12) Filtrado por texto

Filtra el DataFrame `df_productos` para mostrar solo filas cuyo `producto` contenga **camisa** o **pantalón**, ignorando acentos y mayúsculas/minúsculas.

In [181]:
# Copie su código aca
serie_normalizada = (
    df_productos["producto"].fillna("").str.lower().apply(quitar_acentos)
)
df_productos_cam_pan = df_productos[
    serie_normalizada.str.contains(r"\b(camisa|pantalon)\b", regex=True)
]

/tmp/ipykernel_10764/1451716547.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  serie_normalizada.str.contains(r"\b(camisa|pantalon)\b", regex=True)


In [182]:
display(df_productos_cam_pan)

,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.5
1,Pantalón-XL,USD 45,XL,NaN,45.0


## 13) Limpieza de nombres propios

En el DataFrame `df_personas`, crea `nombre_limpio` en `df_personas`:
- Quita paréntesis y su contenido.
- Reemplaza guiones por espacio y elimina espacios extra.
- Aplica *title case* y conserva conectores (`de`, `del`, `la`, `y`) en minúscula.

In [183]:
# Copie su código aca
def normalizar_nombre(nombre: str) -> str:
    if pd.isna(nombre):
        return np.nan
    texto = re.sub(r"\([^)]*\)", "", nombre)
    texto = texto.replace("-", " ")
    texto = " ".join(texto.split())
    conectores = {"de", "del", "la", "y"}
    palabras = []
    for palabra in texto.lower().split():
        base = palabra.lower()
        if base in conectores:
            palabras.append(base)
        else:
            palabras.append(base.capitalize())
    return " ".join(palabras)


df_personas["nombre_limpio"] = df_personas["nombre"].apply(normalizar_nombre)

In [185]:
df_personas["nombre_limpio"]

0      Ana María Lopez
1           Juan Perez
2          Ñandú Gómez
3               Miguel
4    Mónica de la Cruz
5         Luis Delgado
Name: nombre_limpio, dtype: object

## 14) Normaliza respuestas categóricas

En el DataFrame `df_personas`, convierte `categoria` en booleano `categoria_bool` donde cualquier variante de **sí** (con/ sin tilde) sea `True`; el resto `False`.

In [ ]:
# Copie su código aca
serie_categoria = (
    df_personas["categoria"].fillna("").str.strip().str.lower().apply(quitar_acentos)
)
df_personas["categoria_bool"] = serie_categoria.eq("si")

In [ ]:
# df_personas[['nombre','categoria','direccion','id_raw','nombre_limpio','categoria_bool']]

KeyError: "['categoria_bool'] not in index"

## 15) Ciudad desde la dirección

En el DataFrame `df_personas`, Extrae la **ciudad** en una columna `ciudad`

In [35]:
# Copie su código aca


,nombre,categoria,direccion,id_raw,nombre_limpio,categoria_bool,ciudad
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001,Ana María Lopez,True,Bogotá
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001,Juan Perez,True,Lima
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002,Ñandú Gómez,True,Medellín
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003,Miguel,False,Bogotá
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004,Mónica de la Cruz,False,Cali
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005,Luis Delgado,False,CDMX


## 16) Normaliza y valida IDs

En el DataFrame `df_personas`, a partir de `id_raw`, crea la columna `id_norm` con formato `ABC-0001` (tres letras + guion + 4 dígitos) y `id_valido` (True/False).

In [37]:
# Copie su código aca


,id_raw,id_norm,id_valido
0,abc-0001,ABC-0001,True
1,abc 001,ABC-0001,True
2,ABC0002,ABC-0002,True
3,Abc_003,ABC-0003,True
4,abc-00004,ABC-0004,True
5,ABC-0005,ABC-0005,True


## 17) Mini *pipeline* de limpieza general

Escribe una función `limpia_basica(s)` que: ponga el texto en minuscula, quite acentos, URLs, menciones y hashtags, puntuación, dígitos y colapse espacios. Aplícala a `df_texto['texto']` y guarda en `texto_limpio_final`.

In [39]:
# Copie su código aca


,texto,texto_limpio_final
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),hola mundo esto es un ejemplo visita
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el cafe colombiano es buenisimo
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta x en jabon liquido ml codigo a
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",tablas espacios y saltos de linea
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,telefono whatsapp
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,direccion cll medellin barrio la america
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros


## 18)¿Qué tanto cambió el texto?

En el dataframe `df_texto`, 

Cual es la diferencia entre la longitud original (`len_raw`) y la longitud tras la limpieza (`len_clean`)?

In [41]:
# Copie su código aca


,texto,texto_limpio_final,len_raw,len_clean
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),hola mundo esto es un ejemplo visita,84,36
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com,48,30
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el cafe colombiano es buenisimo,66,40
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta x en jabon liquido ml codigo a,52,37
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",tablas espacios y saltos de linea,48,33
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,telefono whatsapp,56,17
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,direccion cll medellin barrio la america,54,40
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros,42,27


### Guardar de resultados

Guarda el dataframe df_texto en parquet con `df_texto.to_parquet('archivo.parquet', index=False)`.

**Phd. Jose R. Zapata**
- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)